# Create plots to visualize data

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from utils.paths import path_angles
import os
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
import numpy as np

#path_angles = r"C:\00_David_Files\forensic\gait_analysis\angles_new"

# Generate large dataframe


In [4]:
models = ["openpose", "alphapose"]
mocap = pd.read_csv("results_mocap_summarized.csv")

# Create an empty list to collect rows
rows = []

# Loop through each model
for model in models:
    # Get all participant directories that start with "P"
    participants = sorted([i for i in os.listdir(os.path.join(path_angles, model)) if i.startswith("P")])
    participants = [p for p in participants if p in mocap["Trial"].values]  # Only include participants that have mocap data
    for participant in participants:
        trial_path = os.path.join(path_angles, model, participant, "summary.csv")
        angles = pd.read_csv(trial_path, index_col=0)  # Assuming row labels are "mean", "std", etc.
        
        # Get the mocap reference values for this participant (if needed for reference)
        mocap_participant = mocap[mocap["Trial"] == participant]
        
        for angle in mocap_participant.columns:
            if angle != "Trial":
                rows.append({
                    "Participant": participant,
                    "Model": model,
                    "Angle": angle,
                    "Mean": angles.loc["mean", angle],
                    "Std": angles.loc["std", angle],
                    "Min": angles.loc["min", angle],
                    "Max": angles.loc["max", angle]
                })

# Add Mocap Reference Values
for _, row in mocap.iterrows():
    participant = row["Trial"]
    for angle in mocap.columns[1:]:
        rows.append({
            "Participant": participant,
            "Model": "mocap",
            "Angle": angle,
            "Mean": row[angle],  # Mocap mean value
            "Std": None,
            "Min": None,
            "Max": None
        })

# Create the final DataFrame from the list of dictionaries
df = pd.DataFrame(rows)

# Save the tidy results to a CSV file
df.to_csv("tidy_results.csv", index=False)

# 1. Comparison between participants

In [20]:
# Load the tidy data (adjust the path if needed)
df = pd.read_csv("tidy_results.csv")

# Create the "plots" directory if it doesn't exist
if not os.path.exists("plots"):
    os.makedirs("plots")

# Get the unique angles and participants
angles_list = df["Angle"].unique()
participants = sorted(df["Participant"].unique())

# Define model colors and bar width
color_alphapose = "blue"
color_openpose = "green"
color_mocap = "red"
bar_width = 0.2  # width for positioning the model plots

# Create a mapping for participant positions on the x-axis
x_positions = {p: i for i, p in enumerate(participants)}

# Loop over each angle to generate a plot
for angle in angles_list:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # For each participant, plot the model and mocap data
    for p in participants:
        x = x_positions[p]
        # Filter rows for current participant and angle (should be three: alphapose, openpose, mocap)
        subset = df[(df["Participant"] == p) & (df["Angle"] == angle)]
        
        for _, row in subset.iterrows():
            model = row["Model"]
            if model in ["alphapose", "openpose"]:
                # Determine x offset and color based on model
                if model == "alphapose":
                    x_offset = x - bar_width/2
                    color = color_alphapose
                else:  # openpose
                    x_offset = x + bar_width/2
                    color = color_openpose
                
                mean_val = row["Mean"]
                std_val = row["Std"]
                min_val = row["Min"]
                max_val = row["Max"]
                
                # Draw the rectangle for ± std (std box)
                if pd.notnull(std_val):
                    std_box_bottom = mean_val - std_val
                    std_box_height = 2 * std_val
                    rect = Rectangle((x_offset - bar_width/2, std_box_bottom),
                                     bar_width,
                                     std_box_height,
                                     alpha=0.3,
                                     color=color)
                    ax.add_patch(rect)
                
                # Draw whiskers: vertical lines from min to (mean - std) and from (mean + std) to max
                ax.vlines(x_offset, min_val, mean_val - std_val, color=color, linewidth=2)
                ax.vlines(x_offset, mean_val + std_val, max_val, color=color, linewidth=2)
                
                # Draw horizontal caps at min and max (short lines)
                cap_width = bar_width / 2
                ax.hlines(min_val, x_offset - cap_width/2, x_offset + cap_width/2, color=color, linewidth=2)
                ax.hlines(max_val, x_offset - cap_width/2, x_offset + cap_width/2, color=color, linewidth=2)
                
                # Optionally, you can also draw a horizontal line at the mean if you want to emphasize it.
                ax.hlines(mean_val, x_offset - bar_width/2, x_offset + bar_width/2, color=color, linewidth=2)
            
            elif model == "mocap":
                # For mocap, draw a red horizontal line spanning the two model positions (for comparison)
                ax.hlines(row["Mean"], x - bar_width, x + bar_width, color=color_mocap, linewidth=2)
    
    # Configure x-axis with participant labels
    ax.set_xticks(list(x_positions.values()))
    ax.set_xticklabels(list(x_positions.keys()))
    
    # Set labels, title, and grid
    ax.set_xlabel("Participant")
    ax.set_ylabel(f"{angle} (degrees)")
    ax.set_title(f"Comparison of {angle} Across Participants")
    ax.grid(True, linestyle="--", alpha=0.7)
    
    # Create a custom legend
    legend_elements = [
        Line2D([0], [0], color=color_alphapose, lw=4, label="Alphapose"),
        Line2D([0], [0], color=color_openpose, lw=4, label="Openpose"),
        Line2D([0], [0], color=color_mocap, lw=4, label="Mocap")
    ]
    ax.legend(handles=legend_elements)
    
    # Save the figure into the "plots" directory
    plot_filename = os.path.join("plots_new", f"Comparison_{angle}.png")
    plt.tight_layout()
    plt.savefig(plot_filename)
    # dont plot at all
    plt.close()


# 2. Errorbar Plot

In [21]:
# Load the tidy data (adjust the path if needed)
df = pd.read_csv("tidy_results.csv")

# Create the "plots" directory if it doesn't exist
if not os.path.exists("plots"):
    os.makedirs("plots")

# Pivot the DataFrame so that each Participant/Angle has the mean values for each model.
# This results in columns like "alphapose", "openpose", and "mocap" containing the mean values.
df_pivot = df.pivot_table(index=["Participant", "Angle"], columns="Model", values="Mean")
df_pivot = df_pivot.reset_index()

# Get the unique angles and participants
angles = df_pivot["Angle"].unique()
participants = sorted(df_pivot["Participant"].unique())
x_positions = np.arange(len(participants))

for angle in angles:
    # Select data for the current angle
    subset = df_pivot[df_pivot["Angle"] == angle]
    
    # Initialize lists for differences for each model
    alphapose_diffs = []
    openpose_diffs = []
    
    # Loop over participants to compute differences: (model mean - mocap mean)
    for p in participants:
        row = subset[subset["Participant"] == p]
        if row.empty:
            alphapose_diffs.append(np.nan)
            openpose_diffs.append(np.nan)
        else:
            row = row.iloc[0]
            m_mean = row["mocap"]
            a_mean = row["alphapose"]
            o_mean = row["openpose"]
            alphapose_diffs.append(a_mean - m_mean)
            openpose_diffs.append(o_mean - m_mean)
    
    # For each difference, create a “one-sided” error: the error bar should extend from 0 to the difference.
    # For a positive difference, the lower error is the full difference and upper error is 0.
    # For a negative difference, the lower error is 0 and the upper error is the absolute value of the difference.
    alphapose_yerr = [(diff if diff >= 0 else 0, 0 if diff >= 0 else -diff) for diff in alphapose_diffs]
    openpose_yerr = [(diff if diff >= 0 else 0, 0 if diff >= 0 else -diff) for diff in openpose_diffs]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot alphapose differences with error bars (shift x positions slightly to the left)
    ax.errorbar(x_positions - 0.1, alphapose_diffs, yerr=np.array(alphapose_yerr).T,
                fmt='o', color='blue', capsize=5, label='Alphapose')
    
    # Plot openpose differences with error bars (shift x positions slightly to the right)
    ax.errorbar(x_positions + 0.1, openpose_diffs, yerr=np.array(openpose_yerr).T,
                fmt='o', color='green', capsize=5, label='Openpose')
    
    # Draw a horizontal dashed line at 0 to indicate no difference
    ax.axhline(0, color='red', linestyle='--', linewidth=2)
    
    # Configure x-axis with participant labels
    ax.set_xticks(x_positions)
    ax.set_xticklabels(participants)
    
    ax.set_xlabel("Participant")
    ax.set_ylabel("Difference (Model - Mocap) in degrees")
    ax.set_title(f"Error (Difference) between Pose Estimation and Mocap for {angle}")
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.legend()
    
    # Save the figure into the "plots" directory
    plot_filename = os.path.join("plots_new", f"Errorplot_{angle}.png")
    plt.tight_layout()
    plt.savefig(plot_filename)
    #plt.show()
    plt.close()

# 3. Bland-Altman Plot

In [22]:
# Load the tidy data (adjust the path if needed
df = pd.read_csv("tidy_results.csv")

# Create the "plots" directory if it doesn't exist
if not os.path.exists("plots"):
    os.makedirs("plots")

# Pivot the DataFrame so that each row corresponds to a Participant and Angle
# with columns for the mean values of each model (alphapose, openpose, mocap)
df_pivot = df.pivot_table(index=["Participant", "Angle"], columns="Model", values="Mean").reset_index()

# Get the unique angles and define the models to analyze
angles = df_pivot["Angle"].unique()
models = ["alphapose", "openpose"]

# Loop through each angle and model to create Bland–Altman plots
for angle in angles:
    # Filter for the current angle
    subset = df_pivot[df_pivot["Angle"] == angle]
    
    for model in models:
        # Calculate the average measurement and the difference (model - mocap)
        averages = (subset[model] + subset["mocap"]) / 2
        differences = subset[model] - subset["mocap"]
        
        # Compute mean difference and standard deviation of the differences
        mean_diff = np.mean(differences)
        std_diff = np.std(differences, ddof=1)  # sample standard deviation
        
        # Define limits of agreement: mean difference ± 1.96*std
        loa_upper = mean_diff + 1.96 * std_diff
        loa_lower = mean_diff - 1.96 * std_diff
        
        # Create the Bland–Altman plot
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.scatter(averages, differences, 
                   color='blue' if model == "alphapose" else 'green', 
                   label=f"{model.capitalize()} Data")
        
        # Plot horizontal lines for the mean difference and the limits of agreement
        ax.axhline(mean_diff, color='gray', linestyle='--', linewidth=2, label='Mean Difference')
        ax.axhline(loa_upper, color='red', linestyle='--', linewidth=2, label='Upper LoA')
        ax.axhline(loa_lower, color='red', linestyle='--', linewidth=2, label='Lower LoA')
        
        # Set title, labels, and grid
        ax.set_title(f"Bland–Altman Plot for {model.capitalize()} - {angle}")
        ax.set_xlabel("Average of Model and Mocap Measurements")
        ax.set_ylabel("Difference (Model - Mocap) in Degrees")
        ax.grid(True, linestyle='--', alpha=0.7)
        ax.legend()
        
        # Save the plot
        plot_filename = os.path.join("plots_new", f"BlandAltman_{model}_{angle}.png")
        plt.tight_layout()
        plt.savefig(plot_filename)
        #plt.show()
        plt.close()

# 4. Scatter plot

In [5]:
# Load tidy data (adjust the path if needed)
df = pd.read_csv("tidy_results.csv")

# Pivot the DataFrame so each row corresponds to a Participant and Angle, 
# with columns for each model's mean values (alphapose, openpose, mocap)
df_pivot = df.pivot_table(index=["Participant", "Angle"], columns="Model", values="Mean").reset_index()

# Get unique angles
angles = df_pivot["Angle"].unique()

# Create a "plots" directory if it doesn't exist
if not os.path.exists("plots"):
    os.makedirs("plots")

for angle in angles:
    subset = df_pivot[df_pivot["Angle"] == angle]
    
    # Create a scatter plot: x = mocap, y = pose estimation
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot alphapose data
    ax.scatter(subset["mocap"], subset["alphapose"], color="blue", label="Alphapose", marker="o")
    # Plot openpose data
    ax.scatter(subset["mocap"], subset["openpose"], color="green", label="Openpose", marker="x")
    
    # Determine plotting range
    min_val = min(subset["mocap"].min(), subset["alphapose"].min(), subset["openpose"].min())
    max_val = max(subset["mocap"].max(), subset["alphapose"].max(), subset["openpose"].max())
    
    # Plot the identity line (perfect agreement line)
    ax.plot([min_val, max_val], [min_val, max_val], 'k--', label="Identity Line")
    
    # Fit a regression line for alphapose vs. mocap
    coeffs_a = np.polyfit(subset["mocap"], subset["alphapose"], 1)
    poly_a = np.poly1d(coeffs_a)
    x_range = np.linspace(min_val, max_val, 100)
    ax.plot(x_range, poly_a(x_range), color="blue", linestyle="--",
            label=f"Alphapose Regression (slope={coeffs_a[0]:.2f})")
    
    # Fit a regression line for openpose vs. mocap
    coeffs_o = np.polyfit(subset["mocap"], subset["openpose"], 1)
    poly_o = np.poly1d(coeffs_o)
    ax.plot(x_range, poly_o(x_range), color="green", linestyle="--",
            label=f"Openpose Regression (slope={coeffs_o[0]:.2f})")
    
    # Labels, title, and grid
    ax.set_xlabel("Mocap Angle (degrees)")
    ax.set_ylabel("Pose Estimation Angle (degrees)")
    ax.set_title(f"Scatter Plot of Mocap vs Pose Estimation for {angle}")
    ax.grid(True, linestyle="--", alpha=0.7)
    ax.legend()
    
    # Save the plot
    plot_filename = os.path.join("plots", f"scatter_mocap_vs_pose_{angle}.png")
    plt.tight_layout()
    plt.savefig(plot_filename)
    plt.close()
    #plt.show()
